<div align="center"><img src='ADS_Capstone.png' align="center"></div>

<h1 align="center">Applied Data Science Capstone Project</h1>
<p align="center">This is the Jupyter notebook used on the Applied Data Science Capstone, part of the IBM Data Science Professional Certificate on Coursera.</p>

<h2 align="center">Week 1</h2>

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h2 align="center">Week 2</h2>

I stored my Foursquare credentials in a separate file (.env), added it to .gitignore!pi and I will use the python-dotenv package to read them.  
This way the credentials don't get exposed on GitHub.

In [3]:
%load_ext dotenv
%dotenv
import os
fs_id = os.getenv("client_id")
fs_sec = os.getenv("client_sec")
print('Credentials loaded')

Credentials loaded


<h2 align="center">Week 3</h2>

<h3 align="center">Part 1</h3>

I'll start by scraping the data and transforming it to a pandas dataframe.

In [4]:
df_orig=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df_orig.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


I'll start cleaning the data by removing rows without an assigned borough.

In [5]:
df_clean = df_orig[df_orig.Borough != 'Not assigned']
df_clean.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


If a postal code has a Borough, but a neighbourhood is not assigned, the neighbourhood will be te same as the borough.

In [6]:
pd.options.mode.chained_assignment = None
df_clean.Neighbourhood[df_clean.Neighbourhood == "Not assigned"] = df_clean.Borough

_The assignment asks for grouping the neighbourhoods that share the same postal code. The current version of the table already has them grouped, therefore I will skip this step._

In [7]:
print(f'The dataframe has {df_clean.shape[0]} rows.')

The dataframe has 103 rows.


<h3 align="center">Part 2</h3>

I will now get the coordinates of each postal code.

In [8]:
df_coord = pd.read_csv('https://cocl.us/Geospatial_data')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


The next step is merging the dataframes.

In [9]:
df_comp = pd.merge(df_clean, df_coord, on='Postal Code')
df_comp.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h3 align="center">Part 3</h3>

#### Visualize the Toronto neighbourdhoods on the map 

First lets import folium (for the map) and Nominatim (to get the coordinates for Toronto).

In [10]:
import folium
from geopy.geocoders import Nominatim

Now I'll get the Toronto coordinates and generate the map with markers on the neigbourhoods.

In [11]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [12]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighbourhood in zip(df_comp['Latitude'], df_comp['Longitude'], df_comp['Borough'], df_comp['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

_The above map may not display correctly on github, if that's the case check the html file on the same repository._

For simplicity I'll focus only on neighbourhoods with Toronto on the name.

In [13]:
to_df = df_comp[df_comp['Borough'].str.contains('Toronto')].reset_index(drop=True)
to_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


Here is the map.

In [14]:
to_map = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighbourhood in zip(to_df['Latitude'], to_df['Longitude'], to_df['Borough'], to_df['Neighbourhood']):
    to_label = '{}, {}'.format(neighbourhood, borough)
    to_label = folium.Popup(to_label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=to_label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(to_map)  
    
to_map

_The above map may not display correctly on github, if that's the case check the html file on the same repository._

We don't need the postal code and borough, so let's drop them.

In [15]:
to_df.drop(["Postal Code", "Borough"], axis=1, inplace=True)
to_df

,Neighbourhood,Latitude,Longitude
0,"Regent Park, Harbourfront",43.654260,-79.360636
1,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,"Garden District, Ryerson",43.657162,-79.378937
3,St. James Town,43.651494,-79.375418
4,The Beaches,43.676357,-79.293031
5,Berczy Park,43.644771,-79.373306
6,Central Bay Street,43.657952,-79.387383
7,Christie,43.669542,-79.422564
8,"Richmond, Adelaide, King",43.650571,-79.384568
9,"Dufferin, Dovercourt Village",43.669005,-79.442259


#### Exploring neighbourhoods

I'll start by importing the needed libraries.

In [16]:
import requests
import json

Now I'll create the function to explore the neighbourhoods and call it to create a new dataframe with all the venues.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v=20180605&ll={},{}&radius={}&limit=100'.format(
            fs_id, 
            fs_sec,
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
to_venues = getNearbyVenues(names=to_df['Neighbourhood'],
                                   latitudes=to_df['Latitude'],
                                   longitudes=to_df['Longitude']
                                  )
to_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's analyze each neighbourhood.  
I'll start by using one hot encoding and then grouping the result by neighbourhood by taking the mean of the frequency of occurrence of each category

In [19]:
# One hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
to_onehot['Neighbourhood'] = to_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

# Group by nwighbourhood
to_grouped = to_onehot.groupby('Neighbourhood').mean().reset_index()
to_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.038462
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.000000,0.00,0.016667,0.000000,0.000000,0.016667,0.016667
5,Christie,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012658,0.000000,0.00,...,0.012658,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.025316
7,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
8,Davisville,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


Now we'll get the top venues from each neighbourhood

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = to_grouped['Neighbourhood']

for ind in np.arange(to_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Bakery,Coffee Shop,Bar
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Pizza Place,Auto Workshop,Farmers Market,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Plane
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place


#### Clustering neighbourhoods

Let's cluster the neighbourhoods into 5 clusters.

In [22]:
#We need to take care of the imports
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [23]:
# set number of clusters
kclusters = 5

to_clustering = to_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Now I'll create a new dataframe that combines The neighbourhood, coordinates, cluster and top venues.

In [24]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = to_df

# merge dataframes to add latitude/longitude for each neighborhood
to_merged = to_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

to_merged

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot
1,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Gym,Hobby Shop,Café
2,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop
3,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,Hotel,American Restaurant
4,The Beaches,43.676357,-79.293031,2,Neighborhood,Pub,Health Food Store,Trail,Museum
5,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market
6,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place
7,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Baby Store,Candy Store
8,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant
9,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Bakery,Pharmacy,Pool,Middle Eastern Restaurant,Liquor Store


Let's visualize the clusters

In [25]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighbourhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

_The above map may not display correctly on github, if that's the case check the html file on the same repository._

As we can see, most of the neighbourhoods are on the same cluster wich gives me the impression of high similarity between neighbourhoods. Further analysis would be needed to confirm this.

In [26]:
to_merged[['Neighbourhood', 'Cluster Labels']].groupby('Cluster Labels').count()

,Neighbourhood
Cluster Labels,
0,33
1,3
2,1
3,1
4,1


<h2 align="center">Week 4</h2>

<h2 align="center">Week 5</h2>